In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from PIL import Image, ImageOps

In [ ]:
train_batch = 64
test_batch = 64

learning_rate = 1e-2

epochs = 16

# Device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the training dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=train_batch, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=test_batch, shuffle=False)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_stack(x)
        return output

In [ ]:
def train(
        model: NeuralNetwork,
        optimizer: torch.optim.Optimizer,
        criterion: torch.nn.CrossEntropyLoss,
):
        for epoch in range(epochs):
                for images, labels in train_loader:
                        images, labels = images.to(device), labels.to(device)
                        
                        outputs = model(images)
                        loss = criterion(outputs, labels)

                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                print(f'Epoch [{epoch}/{epochs}] | Loss [{loss.item():.4f}]')

In [ ]:
def evaluate(
        model: NeuralNetwork,
):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy | {100 * correct / total}')

In [ ]:
def evaluate_image(
        model: NeuralNetwork,
        image: Image,
        label: int,
):
        image = image.resize((28,28))
        image = image.convert('L')

        threshold = 128
        image = image.point(lambda x: 255 if x > threshold else 0, '1')

        image = image.convert('L')
        image = ImageOps.invert(image)

        transform = transforms.Compose([
                transforms.ToTensor()
                ])
        
        img_tensor = transform(image).to(device)

        model.eval()
        with torch.no_grad():
                output = model(img_tensor)
                _, predicted = torch.max(output.data, 1)
                print(f'Expected: {label} | Predicted: {predicted.item()}')


In [ ]:
model = NeuralNetwork().to(device)

cross = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

train(model,optimizer,cross)

evaluate(model)

In [ ]:
img = Image.open('test.png')
expected = 1 # The expected number written in test.png

evaluate_image(model,img,expected)